# Text Classification with Deep Learning

## History

### 2020/11/3

- Explain how to fine-tune BERT for text classification with ktrain
- Change `weights` to `embeddings_initializer`
- Call `clear_session()` when creating models in a loop to avoid OOM

**References**
- [Using pre-trained word embeddings | version: Last modified: 2020/05/05](https://keras.io/examples/nlp/pretrained_word_embeddings/)
- [tf.keras.backend.clear_session
](https://www.tensorflow.org/api_docs/python/tf/keras/backend/clear_session)

## Setup

In [ ]:
%tensorflow_version 2.x

In [ ]:
!pip install janome beautifulsoup4

     |████████████████████████████████| 19.7MB 72.6MB/s 


In [ ]:
!mkdir data
!mkdir models
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz -P data/

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘models’: File exists
--2020-11-03 10:34:40--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1279641604 (1.2G) [binary/octet-stream]
Saving to: ‘data/cc.ja.300.vec.gz.1’

cc.ja.300.vec.gz.1    0%[                    ]  11.34M  11.6MB/s               ^C


### Hyper-parameters

In [ ]:
maxlen = 300
num_words = 40000
num_label = 2

### Imports

In [ ]:
import string

import gensim
import numpy as np
import pandas as pd
import tensorflow as tf
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, Embedding, SimpleRNN, LSTM, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

## The dataset

### Load the Amazon Customer Reviews Datasets

In [ ]:
def filter_by_ascii_rate(text, threshold=0.9):
    ascii_letters = set(string.printable)
    rate = sum(c in ascii_letters for c in text) / len(text)
    return rate <= threshold


def load_dataset(filename, n=5000):
    df = pd.read_csv(filename, sep='\t')

    # Converts multi-class to binary-class.
    mapping = {1: 0, 2: 0, 4: 1, 5: 1}
    df = df[df.star_rating != 3]
    df.star_rating = df.star_rating.map(mapping)

    # extracts Japanese texts.
    is_jp = df.review_body.apply(filter_by_ascii_rate)
    df = df[is_jp]

    # sampling.
    df = df.sample(frac=1, random_state=7)  # shuffle
    grouped = df.groupby('star_rating')
    df = grouped.head(n=n)
    return df.review_body.values, df.star_rating.values


url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz'
x, y = load_dataset(url)

### Load the word embeddings

In [ ]:
def load_fasttext(filepath, binary=False):
    """Loads fastText vectors.

    Args:
        filepath (str): a path to a fastText file.

    Return:
        model: KeyedVectors
    """
    model = gensim.models.KeyedVectors.load_word2vec_format(filepath, binary=binary)
    return model


wv = load_fasttext('data/cc.ja.300.vec.gz')

### Preprocess the dataset

In [ ]:
t = Tokenizer(wakati=True)


def build_vocabulary(texts, num_words=None):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=num_words, oov_token='<UNK>'
    )
    tokenizer.fit_on_texts(texts)
    return tokenizer


def clean_html(html, strip=False):
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text(strip=strip)
    return text


def tokenize(text):
    return t.tokenize(text)


def preprocess_dataset(texts):
    texts = [clean_html(text) for text in texts]
    texts = [' '.join(tokenize(text)) for text in texts]
    return texts


def filter_embeddings(embeddings, vocab, num_words, dim=300):
  """Filter word vectors.

  Args:
      embeddings: a dictionary like object.
      vocab: word-index lookup table.
      num_words: the number of words.
      dim: dimension.

  Returns:
      numpy array: an array of word embeddings.
  """
  _embeddings = np.zeros((num_words, dim))
  for word in vocab:
      if word in embeddings:
          word_id = vocab[word]
          if word_id >= num_words:
              continue
          _embeddings[word_id] = embeddings[word]

  return _embeddings

In [ ]:
x = preprocess_dataset(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
vocab = build_vocabulary(x_train, num_words)
x_train = vocab.texts_to_sequences(x_train)
x_test = vocab.texts_to_sequences(x_test)
x_train = pad_sequences(x_train, maxlen=maxlen, truncating='post', padding='post')
x_test = pad_sequences(x_test, maxlen=maxlen, truncating='post', padding='post')

wv = filter_embeddings(wv, vocab.word_index, num_words)

## The models

### Build the models

In [ ]:
class RNNModel:

    def __init__(self, input_dim, output_dim,
                 emb_dim=300, hid_dim=100,
                 embeddings=None, trainable=True):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       mask_zero=True,
                                       trainable=trainable,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       mask_zero=True,
                                       trainable=trainable,
                                       embeddings_initializer=tf.keras.initializers.Constant(embeddings),
                                       # weights=[embeddings],
                                       name='embedding')
        self.rnn = SimpleRNN(hid_dim, name='rnn')
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        output = self.rnn(embedding)
        y = self.fc(output)
        return Model(inputs=x, outputs=y)


class LSTMModel:

    def __init__(self, input_dim, output_dim,
                 emb_dim=300, hid_dim=100,
                 embeddings=None, trainable=True):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       mask_zero=True,
                                       trainable=trainable,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       mask_zero=True,
                                       trainable=trainable,
                                       embeddings_initializer=tf.keras.initializers.Constant(embeddings),
                                       # weights=[embeddings],
                                       name='embedding')
        self.lstm = LSTM(hid_dim, name='lstm')
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        output = self.lstm(embedding)
        y = self.fc(output)
        return Model(inputs=x, outputs=y)


class CNNModel:

    def __init__(self, input_dim, output_dim,
                 filters=250, kernel_size=3,
                 emb_dim=300, embeddings=None, trainable=True):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       trainable=trainable,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       trainable=trainable,
                                       embeddings_initializer=tf.keras.initializers.Constant(embeddings),
                                       # weights=[embeddings],
                                       name='embedding')
        self.conv = Conv1D(filters,
                           kernel_size,
                           padding='valid',
                           activation='relu',
                           strides=1)
        self.pool = GlobalMaxPooling1D()
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        conv = self.conv(embedding)
        pool = self.pool(conv)
        y = self.fc(pool)
        return Model(inputs=x, outputs=y)

In [ ]:
models = [
    RNNModel,
    LSTMModel,
    CNNModel,
    CNNModel
]

### Train the models

In [ ]:
model_path = 'models/model_{}'
embeddings = [None, None, None, wv]
batch_size = 128
epochs = 100
i = 0
for model, embedding in zip(models, embeddings):
    tf.keras.backend.clear_session()
    model = model(num_words, num_label, embeddings=embedding).build()
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['acc']
    )

    callbacks = [
        EarlyStopping(patience=3),
        ModelCheckpoint(model_path.format(i), save_best_only=True)
    ]

    model.fit(
        x=x_train, y=y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        callbacks=callbacks,
        shuffle=True
    )
    i += 1

Epoch 1/100
50/50 [==============================] - 20s 391ms/step - loss: 0.6696 - acc: 0.5814 - val_loss: 0.6423 - val_acc: 0.6331
Epoch 2/100
50/50 [==============================] - 19s 372ms/step - loss: 0.3559 - acc: 0.8641 - val_loss: 0.5793 - val_acc: 0.7200
Epoch 3/100
50/50 [==============================] - 17s 338ms/step - loss: 0.0606 - acc: 0.9841 - val_loss: 0.6690 - val_acc: 0.7344
Epoch 4/100
50/50 [==============================] - 17s 333ms/step - loss: 0.0044 - acc: 1.0000 - val_loss: 0.7889 - val_acc: 0.7113
Epoch 5/100
50/50 [==============================] - 17s 335ms/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.7995 - val_acc: 0.7231
Epoch 1/100
50/50 [==============================] - 18s 364ms/step - loss: 0.5595 - acc: 0.7089 - val_loss: 0.4573 - val_acc: 0.7850
Epoch 2/100
50/50 [==============================] - 9s 172ms/step - loss: 0.2631 - acc: 0.8981 - val_loss: 0.4654 - val_acc: 0.8100
Epoch 3/100
50/50 [==============================] - 9s 170ms/s

### Evaluate the models

In [ ]:
class InferenceAPI:
    """A model API that generates output sequence.

    Attributes:
        model: Model.
        vocab: language's vocabulary.
    """

    def __init__(self, model, vocab, preprocess):
        self.model = model
        self.vocab = vocab
        self.preprocess = preprocess

    def predict_from_texts(self, texts):
        x = self.preprocess(texts)
        x = self.vocab.texts_to_sequences(x)
        return self.predict_from_sequences(x)

    def predict_from_sequences(self, sequences):
        sequences = pad_sequences(sequences, truncating='post')
        y = self.model.predict(sequences)
        return np.argmax(y, -1)

In [ ]:
model_names = ['RNN', 'LSTM', 'CNN', 'CNN(wv)']
for i, model_name in enumerate(model_names):
    tf.keras.backend.clear_session()
    model = load_model(model_path.format(i))
    api = InferenceAPI(model, vocab, preprocess_dataset)
    y_pred = api.predict_from_sequences(x_test)
    print(model_name)
    print('precision\t: {:.4f}'.format(precision_score(y_test, y_pred, average='binary')))
    print('recall\t: {:.4f}'.format(recall_score(y_test, y_pred, average='binary')))
    print('f1\t: {:.4f}'.format(f1_score(y_test, y_pred, average='binary')))
    print()

RNN
precision	: 0.7519
recall	: 0.6864
f1	: 0.7177

LSTM
precision	: 0.7471
recall	: 0.8908
f1	: 0.8126

CNN
precision	: 0.8451
recall	: 0.8417
f1	: 0.8434

CNN(wv)
precision	: 0.8321
recall	: 0.8687
f1	: 0.8500



# Fine-tune BERT for Text Classification with ktrain

文量の都合上、書籍には含まれていませんが、東北大が公開している日本語のBERTをFine-tuneして文章分類をしてみましょう。現在では、BERTを使う場合、Huggingfaceの[Transformers](https://github.com/huggingface/transformers)というパッケージが使われることが多いですが、ここではよりシンプルに書ける[ktrain](https://github.com/amaiya/ktrain)を使ってみます。細かい設定はともかく、サクッと学習させたいときに便利です。

## Setup

MeCabとktrainをインストールします。MeCabはBERTのTokenizerの中で使われています。

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev -y
!pip install mecab-python3 fugashi ipadic
!pip install ktrain

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

### Hyper-parameters

In [ ]:
maxlen = 300
lr = 2e-5
epochs = 2
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

### Imports

In [ ]:
import ktrain
from ktrain import text
from sklearn.metrics import classification_report

## The dataset

### Preprocess the dataset

前処理としては、HTMLタグの除去とテキストをBERTに入力できる形式に変換しています。`text.Transformer`を使うことで、HuggingfaceのTransformersをラップして利用することができます。

In [ ]:
x = [clean_html(text) for text in x]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

In [ ]:
classes = list(set(y_train))
t = text.Transformer(MODEL_NAME, maxlen=maxlen, class_names=classes)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_train(x_val, y_val)


preprocessing train...
language: ja


Is Multi-Label? False
preprocessing train...
language: ja


Is Multi-Label? False


## The models

### Build the model

In [ ]:
model = t.get_classifier()

### Train the model

In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=8)
learner.fit_onecycle(lr, epochs)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
750/750 [==============================] - 815s 1s/step - loss: 0.4033 - accuracy: 0.8153 - val_loss: 0.3108 - val_accuracy: 0.8820
Epoch 2/2
750/750 [==============================] - 814s 1s/step - loss: 0.2030 - accuracy: 0.9302 - val_loss: 0.2744 - val_accuracy: 0.8975


### Evaluate the model

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
y_pred = predictor.predict(x_test)

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9119    0.9092    0.9105      1002
           1     0.9091    0.9118    0.9105       998

    accuracy                         0.9105      2000
   macro avg     0.9105    0.9105    0.9105      2000
weighted avg     0.9105    0.9105    0.9105      2000



### Save and Load the model

In [ ]:
predictor.save('/tmp/model')

In [ ]:
reloaded_predictor = ktrain.load_predictor('/tmp/model')

In [ ]:
x_test[1]

'この映画は「観る」というよりは、「眺める」ほうが向いている作品です。なんといっても内容がつまらないですこの監督は人間よりも犬の方が好きなんだなあと思いました。関係ないですが映像をほめているのになぜ１つ星かというと・・・映像のきれいさなんていうのは、技術とともに上がってくるものなので、いまはこの映像はきれいだなあなんて感心してしまいますが、何年かたったら、まあまあじゃないってなくらいになってるんじゃないかなあ、、でも映画の内容が良ければ、いつの時代でも見れますものね、、ってなわけで１です'

In [ ]:
reloaded_predictor.predict(x_test[1])

0

# Text Classification with Universal Sentence Encoder on TensorFlow Hub

この例では、[TensorFlow Hub](https://tfhub.dev/)を使って文書分類をしてみましょう。TensorFlow Hubとは何かというと、機械学習モデルのリポジトリです。より噛み砕いて言うと、画像分類やテキスト分類、音声認識といったタスクに使うことのできる学習済みのモデルが置いてある場所です。TensorFlow Hubから学習済みのモデルをダウンロードすることで、そのモデルを基に自分のモデルを構築することができます。

今回は、文書分類のために、Multilingual Universal Sentence Encoder(m-USE)と呼ばれるモデルを利用することにしましょう。m-USEを使うことで、多言語の文表現を得られます。モデルの詳細は以下から確認することができます。

- [Multilingual Universal Sentence Encoder | TensorFlow Hub](https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3)

## Setup

m-USEは、入力の処理を[TensorFlow Text](https://github.com/tensorflow/text)に依存しています。そのため、事前にTensorFlow Textをインストールしておく必要があります。

In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 2.6MB 4.5MB/s 


### Imports

In [ ]:
import string

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
from bs4 import BeautifulSoup
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import load_model, Model

### Resource

TensorFlow Hubを使って、m-USEをKerasのレイヤーとしてロードします。`trainable`パラメータによって、学習する際に重みを更新するか否かを決定します。今回はm-USEの重みも更新することにします。

In [ ]:
model_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'
use_layer = hub.KerasLayer(model_url, trainable=True)

## The dataset

### Preprocess the dataset

前処理としては以下の2つを行います。
- HTMLの除去
- テキストの切り詰め

本来はトークン数でテキストを切り詰めるべきだと思いますが、今回は文字数で切り詰めてしまいます。m-USEのインターフェースの都合上、トークン数で切り詰めるのが容易ではなさそうなためです。

In [ ]:
x = [clean_html(text) for text in x]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
def create_input(input_strings, max_seq_length):
    input_texts = [text[:max_seq_length] for text in input_strings]
    return input_texts

In [ ]:
max_seq_length = 300
train_inputs = create_input(x_train, max_seq_length=max_seq_length)
validation_inputs = create_input(x_test, max_seq_length=max_seq_length)

## The model

### Build the model

In [ ]:
def get_model(use_layer, num_labels, rate=0.1):
    input_strings = tf.keras.layers.Input(shape=[], dtype=tf.string)
    pooled_output = use_layer(input_strings)
    pooled_output = tf.keras.layers.Dropout(rate=rate)(pooled_output)
    output = tf.keras.layers.Dense(units=num_labels, activation='softmax')(pooled_output)

    return tf.keras.Model(
                inputs=[input_strings],
                outputs=output
            )


num_labels = 2
model = get_model(
    use_layer,
    num_labels=num_labels
)

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 512)               85213184  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 85,214,210
Trainable params: 85,214,210
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [ ]:
epochs = 100
batch_size = 16
save_path = '/tmp/model'

model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
      x=np.array(train_inputs), y=y_train,
      validation_split=0.2,
      epochs=epochs,
      callbacks=[
                 tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
                 tf.keras.callbacks.ModelCheckpoint(
                     filepath=save_path,
                     monitor='val_loss',
                     save_best_only=True,
                     mode='min'
                 )
      ],
      shuffle=True
)

Epoch 1/100


200/200 [==============================] - ETA: 0s - loss: 0.6866 - accuracy: 0.5569WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


...

200/200 [==============================] - 71s 353ms/step - loss: 0.3706 - accuracy: 0.8536 - val_loss: 0.4084 - val_accuracy: 0.8225
Epoch 13/100
200/200 [==============================] - 45s 226ms/step - loss: 0.3535 - accuracy: 0.8622 - val_loss: 0.4088 - val_accuracy: 0.8281
Epoch 14/100
200/200 [==============================] - 46s 229ms/step - loss: 0.3360 - accuracy: 0.8719 - val_loss: 0.4195 - val_accuracy: 0.8144
Epoch 15/100
200/200 [==============================] - ETA: 0s - loss: 0.3218 - accuracy: 0.8786INFO:tensorflow:Assets written to: /tmp/model/assets


INFO:tensorflow:Assets written to: /tmp/model/assets


200/200 [==============================] - 72s 359ms/step - loss: 0.3218 - accuracy: 0.8786 - val_loss: 0.4021 - val_accuracy: 0.8281
Epoch 16/100
200/200 [==============================] - ETA: 0s - loss: 0.3027 - accuracy: 0.8911INFO:tensorflow:Assets written to: /tmp/model/assets


INFO:tensorflow:Assets written to: /tmp/model/assets


200/200 [==============================] - 71s 356ms/step - loss: 0.3027 - accuracy: 0.8911 - val_loss: 0.4010 - val_accuracy: 0.8331
Epoch 17/100
200/200 [==============================] - 45s 226ms/step - loss: 0.2862 - accuracy: 0.8989 - val_loss: 0.4044 - val_accuracy: 0.8263
Epoch 18/100
200/200 [==============================] - 46s 229ms/step - loss: 0.2727 - accuracy: 0.9030 - val_loss: 0.4042 - val_accuracy: 0.8344
Epoch 19/100
200/200 [==============================] - 46s 231ms/step - loss: 0.2579 - accuracy: 0.9133 - val_loss: 0.4077 - val_accuracy: 0.8369


### Load the trained model

In [ ]:
model = load_model(save_path)

### Evaluate the model

In [ ]:
y_pred = model.predict(validation_inputs, batch_size=batch_size)

In [ ]:
print(classification_report(y_test, np.argmax(y_pred, axis=-1), digits=4))

              precision    recall  f1-score   support

           0     0.8437    0.8403    0.8420      1002
           1     0.8403    0.8437    0.8420       998

    accuracy                         0.8420      2000
   macro avg     0.8420    0.8420    0.8420      2000
weighted avg     0.8420    0.8420    0.8420      2000

